In [2]:
!pip install findspark


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
!which pip

/home/ubuntu/otus-hw/.venv/bin/pip


In [ ]:
import pandas as pd

from sklearn import datasets

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

iris_data = datasets.load_iris(as_frame='auto')
iris_frame = iris_data.frame

In [1]:
import os

import findspark


findspark.init()
findspark.find()


from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as F

In [2]:
# os.environ['PYSPARK_DRIVER_PYTHON'] = "python"
# os.environ['PYSPARK_PYTHON'] = "../.venv/bin/python"
# os.environ['PYSPARK_PYTHON'] = "/home/ubuntu/otus-hw/.venv/bin/python"

In [3]:
conf = (
    SparkConf().setMaster("yarn").setAppName("data_cleaning")
        .set("spark.executor.memory", "8g")
        .set("spark.driver.memory", "4g")
        .set("spark.sql.execution.arrow.pyspark.enabled", "true")
        .set("spark.executor.instances", 4)
        .set("spark.executor.cores", 2)
)


spark = SparkSession.builder.config(conf=conf).getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-09-15 13:28:41,352 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
HDFS_PATH = "hdfs:///user/data/2019-08-22.txt"

In [5]:
# чтение txt файла и преобразование в df
import re
from pyspark.sql.types import FloatType

rdd = spark.sparkContext.textFile(HDFS_PATH)
header = rdd.take(1)[0]
df = rdd.filter(lambda r: r != header) \
        .map(lambda r: r.split(',')) \
        .toDF(re.sub(r'\s+', '', header).split('|'))

df = df.withColumnRenamed('#tranaction_id', 'tranaction_id')

df = df.withColumn('tx_amount', df['tx_amount'].cast(FloatType()))

In [6]:
df.show()

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|            0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|            1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
|            2|2019-08-22 19:05:33|          0|        753|    35.38|          68733|           0|       0|                0|
|            3|2019-08-22 07:21:33|          0|          0|    80.41|          26493|           0|       0|                0|
|            4|2019-08-22 09:06:17|          1|        981|   102.83|          32777|           0|       0|           

In [29]:
sdf = (
    spark.read
    .format('csv')
    .options(
        header='false', 
        mode="DROPMALFORMED",
        # inferSchema='true', 
        sep=","
    )
    .load(HDFS_PATH)
)

# pdf = sdf.toPandas()

In [30]:
sdf.show()

+--------------------+
|                 _c0|
+--------------------+
|# tranaction_id |...|
+--------------------+



In [20]:
df=spark.read.text(HDFS_PATH)

In [21]:
header=df.first()[0]

In [24]:
df = df.filter(~F.col("value").contains(header)).show(10,False)

+----------------------------------------------+
|value                                         |
+----------------------------------------------+
|0,2019-08-22 06:51:03,0,711,70.91,24663,0,0,0 |
|1,2019-08-22 05:10:37,0,0,90.55,18637,0,0,0   |
|2,2019-08-22 19:05:33,0,753,35.38,68733,0,0,0 |
|3,2019-08-22 07:21:33,0,0,80.41,26493,0,0,0   |
|4,2019-08-22 09:06:17,1,981,102.83,32777,0,0,0|
|5,2019-08-22 18:41:25,3,205,34.20,67285,0,0,0 |
|6,2019-08-22 03:12:21,3,0,47.20,11541,0,0,0   |
|7,2019-08-22 22:36:40,6,809,139.39,81400,0,0,0|
|8,2019-08-22 17:23:29,7,184,87.24,62609,0,0,0 |
|9,2019-08-22 21:09:37,8,931,61.70,76177,0,0,0 |
+----------------------------------------------+
only showing top 10 rows



In [19]:
sdf.show()

+--------------------+
|                 _c0|
+--------------------+
|# tranaction_id |...|
|                   0|
|                   1|
|                   2|
|                   3|
|                   4|
|                   5|
|                   6|
|                   7|
|                   8|
|                   9|
|                  10|
|                  11|
|                  12|
|                  13|
|                  14|
|                  15|
|                  16|
|                  17|
|                  18|
+--------------------+
only showing top 20 rows



In [ ]:
spark.

In [11]:
rdd = spark.sparkContext.textFile("{0}".format(DEFAULT_INPUT_DATA_PATH))

In [ ]:
x = rdd.collect()

In [ ]:
# чтение txt файла и преобразование в df
file_dir = "/user/root/datasets/fraudset/2019-09-21.txt"
import re
from pyspark.sql.types import FloatType

rdd = spark.sparkContext.textFile("{0}".format(file_dir))
header = rdd.take(1)[0]
df = rdd.filter(lambda r: r != header) \
        .map(lambda r: r.split(',')) \
        .toDF(re.sub(r'\s+', '', header).split('|'))

df = df.withColumnRenamed('#tranaction_id', 'tranaction_id')

df = df.withColumn('tx_amount', df['tx_amount'].cast(FloatType()))

print(df.show())

In [13]:
rdd.take(10)

2024-09-15 12:24:49,018 WARN scheduler.TaskSetManager: Lost task 0.0 in stage 3.0 (TID 45, rc1a-dataproc-d-f8s8ei2ipif1urgl.mdb.yandexcloud.net, executor 8): java.io.IOException: Cannot run program "./environment/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:209)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:132)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:105)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:131)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:349)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:313)
	at org.apache.spark.scheduler.ResultTask.runTask

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 48, rc1a-dataproc-d-f8s8ei2ipif1urgl.mdb.yandexcloud.net, executor 8): java.io.IOException: Cannot run program "./environment/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:209)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:132)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:105)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:131)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at 